In [92]:
import numpy as np
import pandas as pd
import os

In [93]:
path = 'data/train.csv'
data = pd.read_csv(path)
data

,id,target,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,0,1.0,-0.098,2.165,0.681,-0.614,1.309,-0.455,-0.236,0.276,...,0.867,1.347,0.504,-0.649,0.672,-2.097,1.051,-0.414,1.038,-1.065
1,1,0.0,1.081,-0.973,-0.383,0.326,-0.428,0.317,1.172,0.352,...,-0.165,-1.695,-1.257,1.359,-0.808,-1.624,-0.458,-1.099,-0.936,0.973
2,2,1.0,-0.523,-0.089,-0.348,0.148,-0.022,0.404,-0.023,-0.172,...,0.013,0.263,-1.222,0.726,1.444,-1.165,-1.544,0.004,0.800,-1.211
3,3,1.0,0.067,-0.021,0.392,-1.637,-0.446,-0.725,-1.035,0.834,...,-0.404,0.640,-0.595,-0.966,0.900,0.467,-0.562,-0.254,-0.533,0.238
4,4,1.0,2.347,-0.831,0.511,-0.021,1.225,1.594,0.585,1.509,...,0.898,0.134,2.415,-0.996,-1.006,1.378,1.246,1.478,0.428,0.253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,0.0,-1.199,0.466,-0.908,2.771,1.631,0.931,0.182,-0.652,...,0.724,0.177,-0.039,0.759,0.461,-0.243,0.525,0.281,-0.255,-1.136
246,246,0.0,0.237,0.233,-0.380,-1.748,0.839,-0.721,-0.114,0.005,...,0.857,0.147,0.601,-0.210,-0.768,1.004,-0.979,0.007,0.112,-0.558
247,247,0.0,1.411,-1.465,0.119,0.583,1.634,-0.207,1.173,1.622,...,-0.499,-0.455,0.759,0.222,0.105,-0.727,0.461,0.760,0.168,-0.719
248,248,1.0,0.620,1.040,0.184,-0.570,-0.087,-0.748,-1.559,-0.553,...,0.557,-1.494,0.977,0.882,-1.512,0.478,-0.910,-0.805,2.029,-0.423


In [94]:
cols = data.shape[1]
train = data.iloc[:,2:cols]
t = np.matrix(train)
X = t[0:230]
X.shape

(230, 300)

In [95]:
t = data.iloc[:,1:2]
t = np.matrix(t)
y = t[0:230]
y.shape

(230, 1)

In [96]:
test = data.iloc[:,2:cols]
t = np.matrix(test)
X2 = t[230:]
X2.shape

(20, 300)

In [97]:
t = data.iloc[:,1:2]
t = np.matrix(t)
y2 = t[230:]
y2.shape

(20, 1)

In [98]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [99]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [100]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [101]:
# initial setup
input_size = 300
hidden_size = 5
num_labels = 2
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((5, 301), (2, 6))

In [102]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((230, 301), (230, 5), (230, 6), (230, 2), (230, 2))

In [103]:
cost(params, input_size, hidden_size, num_labels, X, y, learning_rate)

1.4227252883277939

In [104]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [105]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [106]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate)
J, grad.shape

(1.4227252883277939, (1517,))

In [107]:
a1, z2, a2, z3, h = forward_propagate(X2, theta1, theta2)

In [108]:
h = np.array(h)

for i in h:
    sum = i[0]+i[1]
    i[0] = i[0] / sum
    i[1] = i[1] / sum

h

array([[0.4985042 , 0.5014958 ],
       [0.50575032, 0.49424968],
       [0.50564655, 0.49435345],
       [0.50880423, 0.49119577],
       [0.50336926, 0.49663074],
       [0.50061765, 0.49938235],
       [0.50600302, 0.49399698],
       [0.50330938, 0.49669062],
       [0.50220217, 0.49779783],
       [0.50506001, 0.49493999],
       [0.5042484 , 0.4957516 ],
       [0.50150497, 0.49849503],
       [0.5022628 , 0.4977372 ],
       [0.49859319, 0.50140681],
       [0.4978573 , 0.5021427 ],
       [0.50135686, 0.49864314],
       [0.50442391, 0.49557609],
       [0.50854617, 0.49145383],
       [0.50400837, 0.49599163],
       [0.50055346, 0.49944654]])

In [109]:
y_pred = np.array(np.argmax(h, axis=1))
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])

In [110]:
count = 0

for i in range(y2.size):
    if y2[i] == y_pred[i]:
        count += 1

print(count*100/y2.size)

45.0
